In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
from tqdm.notebook import trange, tqdm

In [2]:
url = 'https://www.linkedin.com/jobs/search/?geoId=106693272&keywords=machine%20learning%20engineer&location=Switzerland'

In [3]:
from src import webscraper

In [4]:
webscraper.WebScraper()

In [3]:
def get_info_di(job, wd, rec, p_limit=2):
        """
        Simple utility function to scrape the various job infos
        from Linkedin
        """
        if rec<p_limit:
            try:
                info_di = {
                    'id' : job.id,
                    'title' : job.find_element_by_css_selector('h3').get_attribute('innerText'),
                    'company_name' : job.find_element_by_css_selector('h4').get_attribute('innerText'),
                    'location' : job.find_element_by_css_selector('[class="job-search'+
                                                                  '-card__location"]').get_attribute('innerText'),
                    'date_posted' : job.find_element_by_css_selector('div>div>time').get_attribute('datetime'),
                    'link' : job.find_element_by_css_selector('a').get_attribute('href'),
                    'description' : job.find_element_by_xpath('/html/body/div[1]/div/section'+
                                                              '/div[2]/section[2]').get_attribute('innerText')
                }

                e_infos = wd.find_element_by_xpath('/html/body/div[1]/div/section/div[2]/section[2]/ul').text.splitlines()
                for i in range(0, len(e_infos), 2):
                    info_di[e_infos[i]] = e_infos[i+1]

                return info_di

            except:
                get_info_di(job, wd, rec+1)
                time.sleep(5)
                
        return {}

In [4]:
def scrape_linkedin(url, verbose=1):
    
    if verbose:
        print('Beginning WebScraping Procedure')

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    wd = webdriver.Chrome('Driver/chromedriver.exe', options=chrome_options)
    wd.get(url)

    s = wd.find_element_by_css_selector('h1>span').get_attribute('innerText')
    no_of_jobs = int(re.sub(r'[^\w]', '', s))

    i = 2
    while i <= int(no_of_jobs/25)+1:
        wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        i = i + 1
        time.sleep(3)
        try:
            wd.find_element_by_xpath('//*[@id="main-content"]/section[2]/button').click()
        except:
            pass

    cookies = wd.find_element_by_xpath('//*[@id="artdeco-global-alert-container"]/div/section/div/div[2]/button[2]').click()
    job_lists = wd.find_element_by_class_name('jobs-search__results-list')
    jobs = job_lists.find_elements_by_tag_name('li') # return a list
    
    if verbose:
        print('Collecting infos')
    
    wd.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)
    job_info = []
    for job in tqdm(jobs):

        job.click() 
        time.sleep(3)
        di = get_info_di(job, wd, rec=0)
        job_info.append(di)
        
    df = pd.DataFrame(job_info)
    
    return df

In [5]:
df = scrape_linkedin(url)

Beginning WebScraping Procedure


WebDriverException: Message: 'chromedriver.exe' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [13]:
df['Employment type'].value_counts()

Full-time     427
Contract       12
Internship      7
Temporary       1
Name: Employment type, dtype: int64

In [19]:
df.to_csv('Data/DataScienceMilano.csv', index=False)